In [23]:
import pandas as pd

In [31]:
UK_to_ISCO_1to1s = pd.read_excel('UK_to_International.xls', header=0, sheetname='1to1s')
UK_to_ISCO_1to2s = pd.read_excel('UK_to_International.xls', header=0, sheetname='1to2 50-50').fillna(method='ffill')
UK_to_ISCO_1to2s_40 = pd.read_excel('UK_to_International.xls', header=0,sheetname='1to2 40-60').fillna(method='ffill')

# Drop the annoying two-to-one entries from UK_to_ISCO_1to1s
UK_null_ix = UK_to_ISCO_1to1s['SOC\n2010'].isnull().nonzero()[0]
UK_null_ix = np.concatenate([UK_null_ix - 1, UK_null_ix])
UK_to_ISCO_1to1s.drop(UK_null_ix, inplace=True)

# Naively merge UK_to_ISCO dataframes
UK_to_ISCO = pd.concat([UK_to_ISCO_1to1s, UK_to_ISCO_1to2s,
                        UK_to_ISCO_1to2s_40], ignore_index=True)

UK_to_ISCO.columns = [[u'Universal'] * UK_to_ISCO.columns.size, UK_to_ISCO.columns]
UK_to_ISCO.columns.names = ['Region', 'Features']

UK_to_ISCO[u'Universal', u'SOC\n2010'] = UK_to_ISCO[u'Universal', u'SOC\n2010'].astype(int)

UK_to_ISCO.head() 

Region   Universal                                                     \
Features    ISCO08                 ISCO08 Unit Group Titles SOC\n2010   
0             1120  Managing directors and chief executives      1115   
1             1111                              Legislators      1116   
2             1321                   Manufacturing managers      1121   
3             1323                    Construction managers      1122   
4             1322                          Mining managers      1123   

Region                                                                  
Features                          SOC2010 Unit Group Titles Unnamed: 4  
0                     Chief executives and senior officials        NaN  
1                      Elected officers and representatives        NaN  
2         Production managers and directors in manufactu...        NaN  
3         Production managers and directors in construction        NaN  
4         Production managers and directors in mining an...        NaN

In [40]:
##########################################################################
# Employment
##########################################################################
# Load in regional employment data
UK_emp = pd.read_excel('empbysocandqualificationsjd13_tcm77-368924.xls',
                       header=7, skip_rows=1, skip_footer=11, na_values=['*', '-'],
                       parse_cols=range(1, 16), sheetname='Emp by occupations and regions')

n_reg = UK_emp.columns.size
# Annoyingly, the default index read contains verbose occupation descriptions
UK_emp['Universal'.decode('utf-8')] = UK_emp.index
UK_emp.index = pd.Index(np.arange(UK_emp.index.size))
UK_emp.drop([0], inplace=True)
UK_emp['Universal'] = UK_emp['Universal'].map(lambda s: int(s[:4]))
UK_emp.fillna(0, inplace=True)

# Do some hierachical indexing
UK_emp.columns = [UK_emp.columns, ['Employment'] * n_reg + ['SOC\n2010'.decode('utf-8')]]
UK_emp.columns.names = ['Region', 'Features']

UK_emp[u'Universal', u'SOC\n2010'] = UK_to_ISCO[u'Universal', u'SOC\n2010'].astype(int)

UK_emp.head()


Region,North East,North West,Merseyside,Yorkshire & Humberside,East Midlands,West Midlands,Eastern,London,South East,South West,Wales,Scotland,Northern Ireland,Total,Universal
Features,Employment,Employment,Employment,Employment,Employment,Employment,Employment,Employment,Employment,Employment,Employment,Employment,Employment,Employment,SOC\n2010
1,621,4241,827,2556,4460,3180,6491,19992,8604,5097,1985,2771,1355,62180,1116
2,269,1465,0,0,1098,0,0,0,1557,1460,527,855,0,8885,1121
3,11260,31887,2704,24008,26571,31498,34087,16631,35510,25140,10057,19835,7609,276797,1122
4,6956,10549,3079,13246,13447,14702,20974,17148,23870,12476,5217,9606,4566,155836,1123
5,519,0,0,1243,0,861,0,0,1238,1689,903,5697,0,14608,1131


In [34]:
print UK_emp.columns.values[-1]
print UK_to_ISCO.columns.values[2]

(u'Universal', u'SOC\n2010')
(u'Universal', u'SOC\n2010')


In [36]:
# Merge in employment data

UK_to_ISCO = pd.merge(UK_to_ISCO, UK_emp, how='inner', on=[u'Universal', u'SOC\n2010'])

KeyError: u'SOC\n2010'